train rect face

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
training_iter=1000
df=pd.read_csv('/content/gdrive/My Drive/faceData/rectFace1.csv')
trainTestData=np.array(df.loc[:,'0':'783']/255)
labelData=np.array(df.loc[:,'xmin':'ymax']/28)
random_no=np.arange(0,4032)
np.random.shuffle(random_no)
x=[]
y=[]
xt=[]
yt=[]
for i in range(4000):
  x.append(trainTestData[random_no[i]])
  y.append(labelData[random_no[i]])
for i in range(4000,4032):
  xt.append(trainTestData[random_no[i]])
  yt.append(labelData[random_no[i]])
x=np.array(x)
y=np.array(y)
xt=np.array(xt)
yt=np.array(yt)
x_train=x#np.reshape(x,(-1,28,28,1))
y_train=y
x_test=xt#np.reshape(xt,(-1,28,28,1))
y_test=yt
X=tf.placeholder(tf.float32,shape=[None,28*28])
Y=tf.placeholder(tf.float32,shape=[None,4])
def conv_layer(X,w,b):
  conv=tf.nn.conv2d(X,w,strides=[1,1,1,1],padding='SAME')
  bias=tf.nn.bias_add(conv,b)
  return tf.nn.relu(bias)
def maxpool_layer(x,k=2):
  maxpool=tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,2,2,1],padding='SAME')
  return maxpool
#w0=tf.Variable(tf.random_normal(shape=[5,5,1,32]),tf.float32)
#w1=tf.Variable(tf.random_normal(shape=[5,5,32,64]),tf.float32)
#w2=tf.Variable(tf.random_normal(shape=[5,5,64,128]),tf.float32)
wd1=tf.Variable(tf.random_normal(shape=[784,128]),tf.float32)
wout=tf.Variable(tf.random_normal(shape=[128,4]),tf.float32)
#b0=tf.Variable(tf.random_normal(shape=[32]),tf.float32)
#b1=tf.Variable(tf.random_normal(shape=[64]),tf.float32)
#b2=tf.Variable(tf.random_normal(shape=[128]),tf.float32)
bd1=tf.Variable(tf.random_normal(shape=[128]),tf.float32)
bout=tf.Variable(tf.random_normal(shape=[4]),tf.float32)
#conv1=conv_layer(X,w0,b0)
#conv1=maxpool_layer(conv1)
#conv2=conv_layer(conv1,w1,b1)
#conv2=maxpool_layer(conv2)
#conv3=conv_layer(conv2,w2,b2)
#conv3=maxpool_layer(conv3)
#fc1=tf.reshape(conv3,[-1,wd1.get_shape().as_list()[0]])
fc1=tf.add(tf.matmul(X,wd1),bd1)
fc1=tf.nn.relu(fc1)
out=tf.add(tf.matmul(fc1,wout),bout)
cost=tf.reduce_sum(tf.square(tf.subtract(out,Y)))
train_op=tf.train.AdadeltaOptimizer(0.01).minimize(cost)
saver=tf.train.Saver()
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_iter):
    for batch in range(0,len(x_train),32):
      batch_x=x_train[batch:batch+32,:]
      batch_y=y_train[batch:batch+32,:]
      sess.run(train_op,feed_dict={X:batch_x,Y:batch_y})
      loss=sess.run(cost,{X:batch_x,Y:batch_y})
    print('iter:',epoch,'loss:',loss)
  saver.save(sess,'/content/gdrive/My Drive/faceData/rect/rect')
  #valid_loss=sess.run(cost,{X:x_test,Y:y_test})    
  #print('testXmin:',valid_loss)
  result=sess.run(out,{X:x_test})
  print(result,y_test)

test rect face

In [9]:
import pandas as pd
import tensorflow as tf
import numpy as np
df=pd.read_csv('//content/drive/My Drive/faceData/rectFace1.csv')
trainTestData=np.array(df.loc[:,'0':'783']/255)
labelData=np.array(df.loc[:,'xmin':'ymax']/28)
random_no=np.arange(0,4032)
np.random.shuffle(random_no)
x=[]
y=[]
xt=[]
yt=[]
for i in range(4000):
  x.append(trainTestData[random_no[i]])
  y.append(labelData[random_no[i]])
for i in range(4000,4032):
  xt.append(trainTestData[random_no[i]])
  yt.append(labelData[random_no[i]])
x=np.array(x)
y=np.array(y)
xt=np.array(xt)
yt=np.array(yt)
x_train=x#np.reshape(x,(-1,28,28,1))
y_train=y
x_test=xt#np.reshape(xt,(-1,28,28,1))
y_test=yt
with tf.Session() as sess:
  saver=tf.train.import_meta_graph('/content/drive/My Drive/faceData/rect/rect.meta')
  saver.restore(sess,'/content/drive/My Drive/faceData/rect/rect')
  graph=tf.get_default_graph()
  X=graph.get_tensor_by_name('Placeholder_2:0')
  Y=graph.get_tensor_by_name('Add_3:0')
  result=sess.run(Y,{X:x_test[0].reshape(1,-1)})
  print(result,y_test[0])

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/faceData/rect/rect
[[0.14550143 0.44813576 0.5546496  0.8688228 ]] [0.1125 0.4025 0.485  0.7775]
